# This is the working script - no edits please

## Set Up

1. In the directory there are seperate folders for each flight containing all the jpegs, and the arw images too
2. Identifiable csv file of the Flight log

## Scripts Contained

1. Takes in all jpgs in a directory, reads their exif data and then creates a new csv file with columns of Image Name / Time Taken / Date Taken
2. Takes in the above csv file, calculates the total seconds elapsed from midnight for each image based off of the time value, then creates a new csv file with the columns Image Name / Time / Seconds Elapsed
3. Takes in the corresponding flight file, and using the startWord = '[L-MIS]WP Mission: received idle vel cmd 2' or endWord = 'turn mode. end of trace. quit' to associate the flight file lines with real times. Using this real time, the total secodns past midnight is calculated. A new csv file is created, with first line having the start and end line outputted, and then the next line having column headers Seconds / Long / Lat / Altitude for every single line in the flight file. This allows for later matching with image capture times that were on misson and not on mission
4. Creates two new csv files, InMissionData and NotInMissionData. Next it reads in both the stripped down flight file from script 3, and the csv file from script 2 and matches each jpg-elapsed second to a corresponding line in the stripped down flight file. Then images which have been identified as being on mission, from using row 1 of the stripped down flight file, have a row written to the InMissionData file which has columns Image Name/Long/Lat/Altitude, and images outwith the mission have a row written to the NotInMissionData file which has one column of Image Name
5. Creates 2 new folders, InMissionData and OutMissionData, and within each folder creates a new folder names jpg and arw. Then by reading in the two csv files from Script 4, sorts the arw and jpg images accordingly to the new folders. Here it is assumed that each arw is directly associated with the same named jpg image


In [34]:
directory = 'D:/ATEC/180717_sample/sony/flight2/'
outputFilename = directory+'jpgImageTimes.csv'

import os
from PIL import Image 
from PIL.ExifTags import TAGS

fileHolder = []
fileCounter = 0
for file in os.listdir(directory):
    if file.endswith(".JPG"):
        fileHolder.append(os.path.join(directory, file))
        fileCounter = fileCounter + 1
        
with open(outputFilename, 'w') as outputFile:
    firstRow = "Image Name" + "," + "Time Taken" + "," + "Date Taken" + "\n"
    outputFile.write(firstRow)
    for i in range(fileCounter):
        image = Image.open(fileHolder[i])
        info = image._getexif() 
        cnt = 0
        for tag, value in info.items():
            cnt = cnt + 1
            if cnt == 4:
                key = TAGS.get(tag, tag) 
                #print(key + " " + str(value))
                row = fileHolder[i] + "," + str(value[-8:]) + "," + str(value[0:10]) + "\n"
                #date = str(value[8:10]) + str(value[4:8]) + str(value[0:4])
                #row = fileHolder[i] + "," + str(value[-8:]) + "," + date + "\n"
                outputFile.write(row)
                
image.close()
print("All done :)")

All done :)


In [35]:
#flightTriggerTime = 120

with open(directory+'jpgImageTimes.csv') as jpgFile:
    jpgFileLines = jpgFile.read().split("\n")
    jpgjpgImageNames = [(jpgFileLines[x]).split(',')[0] for x in range(0, len(jpgFileLines)-1)]
    jpgImageTimes = [(jpgFileLines[x]).split(',')[1] for x in range(0, len(jpgFileLines)-1)]
       
def howManySecs(clockFace):
    secs = int(clockFace[-2:]) + 60*int(clockFace[3:5]) + 60*60*int(clockFace[:2])
    return secs

jpgImageSeconds = []
for j in range(1, len(jpgFileLines)-1):
    jpgImageSeconds.append(howManySecs(jpgImageTimes[j])-3600)

with open(directory+'jpgImageTimesFirstScript.csv', 'w') as jpgFile:
    firstRow = jpgjpgImageNames[0] + "," + jpgImageTimes[0] + "," + "Seconds" + "\n"
    jpgFile.write(firstRow)
    secondRow = jpgjpgImageNames[1] + "," + jpgImageTimes[1] + "," + str(jpgImageSeconds[0]) + "\n"
    jpgFile.write(secondRow)
    for i in range(1, len(jpgFileLines)-2):
        if jpgImageSeconds[i] - jpgImageSeconds[i-1] < flightTriggerTime:
            row = jpgjpgImageNames[i+1] + "," + jpgImageTimes[i+1] +  "," + str(jpgImageSeconds[i]) + "\n"
            jpgFile.write(row)
        else:
            row = jpgjpgImageNames[i+1] + "," + jpgImageTimes[i+1] +  "," + str(jpgImageSeconds[i]) + "\n"
            jpgFile.write(row)
        
print("All done :)")        

All done :)


Expect to be given the directory to the 1 flight file we need. Then this will 

In [36]:
flightLogDirectory = 'D:/ATEC/180717_sample/sony/FLY217_20170718_ES_P2.csv'

import os

with open(flightLogDirectory) as flightFile:      
    flightFileLines = flightFile.read().split("\n")

camShotLine = []
word = 'SUCCESSED SIMPLE_SHOT'

for i,line in enumerate(flightFileLines):
    if word in line: # or word in line.split() to search for full words
        camShotLine.append(i)

missionRangeLine = []
startWord = '[L-MIS]WP Mission: received idle vel cmd 2'
endWord = 'turn mode. end of trace. quit'

for i,line in enumerate(flightFileLines):
    if startWord in line: # or word in line.split() to search for full words
        missionRangeLine.append(i)
        missStartLine = i
        print("Word \"{}\" found in line {}".format(startWord, i))

for j,line in enumerate(flightFileLines):
    if endWord in line: # or word in line.split() to search for full words
        missionRangeLine.append(j)
        missEndLine = j
        print("Word \"{}\" found in line {}".format(endWord, j))

potMatch = len(camShotLine)
potMatchLines = [flightFileLines[x] for x in camShotLine]

dateWord = '|#SiF%|2'

times = []

for y in range(potMatch):
    charDateWord = potMatchLines[y][(potMatchLines[y].find(dateWord)+7):((potMatchLines[y].find(dateWord)+15))]
    if potMatchLines[y].find(charDateWord+".0")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".0")+9):((potMatchLines[y].find(charDateWord+".0")+17))])
    elif potMatchLines[y].find(charDateWord+".1")>1:  
        times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".1")+9):((potMatchLines[y].find(charDateWord+".1")+17))])
    elif potMatchLines[y].find(charDateWord+".2")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".2")+9):((potMatchLines[y].find(charDateWord+".2")+17))])

flightTimes = [(flightFileLines[x]).split(',')[1] for x in camShotLine]

missStartFlightTime = (flightFileLines[missStartLine]).split(',')[1]
missFinishFlightTime = (flightFileLines[missEndLine]).split(',')[1]
flightTimeDiffs = [float(missStartFlightTime)-float(flightTimes[0]), float(missFinishFlightTime)-float(flightTimes[0])]
realFlightTimes = [0,0]
for j in range(2):
    timeHolder = (times[0]).split(':')
    for i in range(3):
        timeHolder[i] = float(timeHolder[i])
    timeHolder[2] = timeHolder[2] + flightTimeDiffs[j]
    while timeHolder[2] >= 60:
        timeHolder[2] = timeHolder[2] - 60
        timeHolder[1] = timeHolder[1] + 1
        while timeHolder[1] >= 60:
            timeHolder[1] = timeHolder[1] - 60
            timeHolder[0] = timeHolder[0] + 1
    while timeHolder[2] <= 0:
        timeHolder[2] = timeHolder[2] + 60
        timeHolder[1] = timeHolder[1] - 1
        while timeHolder[1] <= 0:
            timeHolder[1] = timeHolder[1] + 60
            timeholder[0] = timeHolder[0] - 1
    realFlightTimes[j] = timeHolder

offsetTimes = [(flightFileLines[x]).split(',')[1] for x in range(1, len(flightFileLines)-1)]
longitudes = [(flightFileLines[x]).split(',')[3] for x in range(1, len(flightFileLines)-1)]
latitudes = [(flightFileLines[x]).split(',')[4] for x in range(1, len(flightFileLines)-1)]
altitudes = [(flightFileLines[x]).split(',')[10] for x in range(1, len(flightFileLines)-1)]

startSecond = 60*60*realFlightTimes[0][0] + 60*realFlightTimes[0][1] + realFlightTimes[0][2]
endSecond = 60*60*realFlightTimes[1][0] + 60*realFlightTimes[1][1] + realFlightTimes[1][2]
#initialOffsetTime = offsetTimes[missStartLine]
zerothRealTime = startSecond - (float(offsetTimes[missStartLine-1]) - float(offsetTimes[0]))

with open(directory+'jpgFlightFileScript2.csv', 'w') as outputFlightFile:
    firstRow = "Mission Start Line" + "," + str(missStartLine) + "," + "Mission End Line" + "," + str(missEndLine) + "\n"
    outputFlightFile.write(firstRow)
    secondRow = "Seconds" + "," + "Longitude" + "," + "Latitude" + "," + "Altitude" + "\n"
    outputFlightFile.write(secondRow)
    for i in range(0, len(flightFileLines)-2):  #I took out the +/- 20 buffer here, maybe erroneously
        timeInSeconds = float(offsetTimes[i]) - float(offsetTimes[0]) + zerothRealTime
        row =  str(timeInSeconds) + "," + longitudes[i] + "," + latitudes[i] + "," + altitudes[i] + "\n"
        outputFlightFile.write(row)

print("All done :)")

Word "[L-MIS]WP Mission: received idle vel cmd 2" found in line 3941
Word "turn mode. end of trace. quit" found in line 26735
All done :)


In [37]:
with open(directory+'jpgFlightFileScript2.csv', 'r') as inputFlightFile:
        missionLines = inputFlightFile.read().split("\n")
        
with open(directory+'jpgImageTimesFirstScript.csv', 'r') as inputjpgFile:
        jpgLines = inputjpgFile.read().split("\n")
        
missionSeconds = [float((missionLines[x]).split(',')[0]) for x in range(2, len(missionLines)-1)]
intMissionSeconds = [int(missionSeconds[x]) for x in range(len(missionSeconds))]
jpgSeconds = [int((jpgLines[x]).split(',')[2]) for x in range(1, len(jpgLines)-1)]
longitudes = [(missionLines[x]).split(',')[1] for x in range(2, len(missionLines)-1)]
latitudes = [(missionLines[x]).split(',')[2] for x in range(1, len(missionLines)-1)]
altitudes = [(missionLines[x]).split(',')[3] for x in range(1, len(missionLines)-1)]
jpgImageNames = [(jpgLines[x]).split(',')[0] for x in range(1, len(jpgLines)-1)]

cnt = 0
matched = 0
howManyMatched = -1
jpgSecondsLineNum = []

#will find 46 in 746 btw so be carfeul!!!!!
for sec in jpgSeconds:
    cnt = 0
    while cnt==0:
        try:
            jpgSecondsLineNum.append(intMissionSeconds.index(sec))
            break
        except ValueError:
            print(sec, "doesn't match any of our records, sorry")
            cnt = cnt + 1
            jpgSecondsLineNum.append(intMissionSeconds.append(0))            

print(jpgSecondsLineNum) 
    
with open(directory+'jpgImageFinalMisisonDataScript3.csv', 'w') as jpgFinalDataFile:
    firstRow = "File Name" + "," + "Longitude" + "," + "Latitude" + "," + "Altitude" + "\n"
    jpgFinalDataFile.write(firstRow)
    for i in range(len(jpgSeconds)):
        lineNum = jpgSecondsLineNum[i]
        if type(lineNum) == int or type(lineNum) == float:
            if lineNum >= missStartLine and lineNum <= missEndLine:
                row = jpgImageNames[i] + "," + longitudes[lineNum] + "," + latitudes[lineNum] + "," + altitudes[lineNum] + "\n"
                jpgFinalDataFile.write(row)
            
with open(directory+'jpgImageFinalNotMisisonDataScript3.csv', 'w') as jpgFinalDataFile:
    firstRow = "File Name" + "\n"
    jpgFinalDataFile.write(firstRow)
    for i in range(len(jpgSeconds)):
        lineNum = jpgSecondsLineNum[i]
        if type(lineNum) == int or type(lineNum) == float:
            if lineNum <= missStartLine or lineNum >= missEndLine:
                row = jpgImageNames[i] + "\n"
                jpgFinalDataFile.write(row)
        if type(lineNum) != int and type(lineNum) != float:
            row = jpgImageNames[i] + "\n"
            jpgFinalDataFile.write(row)
            
print("All done :)")

(63653, "doesn't match any of our records, sorry")
[916, 1004, 1063, 1122, 1181, 1239, 1298, 1386, 1445, 1504, 1592, 1680, 1856, 1914, 1973, 2032, 2149, 2207, 2266, 2324, 2412, 2471, 2529, 2606, 2664, 2753, 2812, 2871, 2930, 2989, 3048, 3107, 3166, 3225, 3284, 3343, 3432, 3491, 3550, 3609, 3668, 3727, 3787, 3875, 3934, 4023, 4082, 4171, 4200, 4259, 4318, 4378, 4437, 4496, 4555, 4614, 4673, 4732, 4791, 4880, 4968, 5057, 5116, 5204, 5293, 5382, 5441, 5529, 5618, 5707, 5766, 5854, 5943, 6002, 6091, 6179, 6268, 6327, 6415, 6504, 6592, 6681, 6740, 6829, 6917, 6976, 7065, 7124, 7212, 7301, 7390, 7449, 7537, 7626, 7715, 7774, 7862, 7951, 8010, 8099, 8187, 8276, 8335, 8424, 8512, 8601, 8660, 8749, 8838, 8897, 8986, 9074, 9163, 9252, 9311, 9399, 9488, 9547, 9636, 9725, 9813, 9872, 9961, 10050, 10109, 10198, 10286, 10345, 10434, 10523, 10582, 10671, 10759, 10848, 10907, 10995, 11084, 11143, 11232, 11320, 11409, 11468, 11557, 11645, 11734, 11793, 11882, 11970, 12029, 12118, 12207, 12266, 12347, 1

Now we want to refolder our directory into mission and not mission data

In [38]:
import os

jpgHolder = []
jpgCounter = 0
for file in os.listdir(directory):
    if file.endswith(".jpg"):
        jpgHolder.append(os.path.join(directory, file))
        jpgCounter = jpgCounter + 1
        
arwHolder = []
arwCounter = 0
for file in os.listdir(directory):
    if file.endswith(".arw"):
        arwHolder.append(os.path.join(directory, file))
        arwCounter = arwCounter + 1
        
newpath1 = directory + 'MissionData/jpg/'  
newpath2 = directory + 'NotMissionData/jpg/'
newpath3 = directory + 'MissionData/arw/'  
newpath4 = directory + 'NotMissionData/arw/' 
if not os.path.exists(newpath1):
    os.makedirs(newpath1)
if not os.path.exists(newpath2):
    os.makedirs(newpath2)
if not os.path.exists(newpath3):
    os.makedirs(newpath3)
if not os.path.exists(newpath4):
    os.makedirs(newpath4)
        
with open(directory+'jpgImageFinalNotMisisonDataScript3.csv', 'r') as jpgFinalDataFile:
    jpgLines = jpgFinalDataFile.read().split("\n")
    jpgImageNames = [(jpgLines[x]).split(',')[0] for x in range(1, len(jpgLines)-1)]
    arwImageNames = []
    for z in range(len(jpgImageNames)):
        arwImageNames.append(jpgImageNames[z][:-3] + "ARW")   
    tempImage = (jpgImageNames[0]).split('/')
    fileLocal = len(tempImage)
    jpgImageFileNames = [(jpgImageNames[x]).split('/')[fileLocal-1] for x in range(len(jpgImageNames))]
    arwImageFileNames = [(arwImageNames[x]).split('/')[fileLocal-1] for x in range(len(jpgImageNames))]
    for i in range(len(jpgImageNames)):
        os.rename(jpgImageNames[i],  newpath2 + jpgImageFileNames[i])
        os.rename(arwImageNames[i],  newpath4 + arwImageFileNames[i])
    
with open(directory+'jpgImageFinalMisisonDataScript3.csv', 'r') as jpgFinalDataFile:
    jpgLines = jpgFinalDataFile.read().split("\n")
    jpgImageNames = [(jpgLines[x]).split(',')[0] for x in range(1, len(jpgLines)-1)]
    arwImageNames = []
    for z in range(len(jpgImageNames)):
        arwImageNames.append(jpgImageNames[z][:-3] + "ARW")
    tempImage = (jpgImageNames[0]).split('/')
    fileLocal = len(tempImage)
    jpgImageFileNames = [(jpgImageNames[x]).split('/')[fileLocal-1] for x in range(len(jpgImageNames))]
    arwImageFileNames = [(arwImageNames[x]).split('/')[fileLocal-1] for x in range(len(jpgImageNames))]
    for i in range(len(jpgImageNames)):
        os.rename(jpgImageNames[i],  newpath1 + jpgImageFileNames[i])
        os.rename(arwImageNames[i],  newpath3 + arwImageFileNames[i]) 
        
print("All done :)")

All done :)
